# Homework 6  Backward propagation for CNN

#### Welcome to the course AI and Deep Learning!

Convolutional Neural Networks (CNNs) are a class of deep learning models mainly used for image and spatial data processing. They automatically learn to detect important features such as edges, textures, and shapes by applying convolutional filters over input data.Backpropagation is the training algorithm used to adjust weights in a CNN.It works by:

1.Forward pass – computes the output and loss based on current weights.

2.Backward pass – calculates gradients of the loss with respect to each parameter using the chain rule.(Here you should pay attention to the difference between parameter that needs to update and hyperparameter that needs to select)

3.Parameter update – applies gradient descent (or variants) to minimize the loss.

This process is repeated over many iterations to train the CNN to make accurate predictions.



#### Learning Goal:  In this homework, we aim to understand and implement the complete structure of Convolutional Neural Networks (CNNs),which includes convolution layers,pooling layer.The backward propagation is also clearly illustrated and you can write your code to check how gradients are computed through each layer.You are expected to clearly illustrate the mathematical steps of backpropagation and write corresponding Python code (from scratch or using NumPy) to solidify your understanding.



# 1- Structure

$[Input Image] → [Conv2D] → [Activation] → [Pooling]→[Flatten]→[Fully - Connected - layer]→Output$

# 2- Forward Propagation

### 2.1CNN Forward Propagation Process(multiple samples)
Below is a general example for multiple samples, you can also define N=1 to review the one-sample senario for simplicity.

#### A. Input Layer
- **Input Data**: Assume the input is $Z^{l-1}$ with dimensions $(N, C^{l-1}, H^{l-1}, W^{l-1})$, where:
  - $N$ is the batch size
  - $C^{l-1}$ is the number of channels (3 for RGB)
  - $H^{l-1}$ is the height
  - $W^{l-1}$ is the width
  - e.g.（MNIST）：\( (1, 1, 28, 28) \)

#### B. Convolutional Layer

| parameters       | dimension                | details                  |
|----------------|--------------------------|--------------------------|
| **weights**       | $(C^{l}, C^{l-1}, K_h, K_w)$ | $K_h \times K_w$为kernel size |
| **bias**       | $(C^{l})$              | one bias each output channel       |
| **stride**       | $(S_h, S_w)$             | default(1,1)               |
| **padding**       | $(P_h, P_w)$             | default(0,0)               |


$$
\begin{aligned}
H^{l} &= \left\lfloor \frac{H^{l-1} + 2P_h - K_h}{S_h} + 1 \right\rfloor \\
W^{l} &= \left\lfloor \frac{W^{l-1} + 2P_w - K_w}{S_w} + 1 \right\rfloor
\end{aligned}
$$

**output dimension**：$(N, C^{l}, H^{l}, W^{l})$


**example**：
- input：\( (1, 1, 28, 28) \)
- parameters：\$( C^{(l)}=32 \), \( K_h=K_w=3 \), \( S_h=S_w=1 \), \( P_h=P_w=1 \)$
  kernel $W_{l}:(32,1,3,3)$
  bias $b_{l}；（32）$
- output：\( (1, 32, 28, 28) \)





#### C. Activation Layer (ReLU)
- **Operation**:
  $$\text{ReLU}(x) = \max(0, x)$$
- **Output**:$A^{l}$ Same as input dimensions $(N, C_{out}, H_{\text{out}}, W_{\text{out}})$

#### D. Pooling Layer (MaxPool)
| parameters       | dimension         | details|
|----------------|------------------|-------------|
| **pooling**   | $(P_h, P_w)$     | default(2,2)   |
| **stride**       | $(S_h, S_w)$     | 通常等于窗口大小 |
| **padding**       | $(P_{h}, P_{w})$ | default(0,0) |


$$
\begin{aligned}
H_{out} &= \left\lfloor \frac{H^{l} - P_h}{S_h} + 1 \right\rfloor \\
W_{out} &= \left\lfloor \frac{W^{l} - P_w}{S_w} + 1 \right\rfloor
\end{aligned}
$$

**output dimension**：$Z^{l},dimension(N, C^{l}, H^{out}, W^{out})$
**example**：
- input：\( (1, 32, 28, 28) \)
- output：\( (1, 32, 14, 14) \)

#### E. Flatten
- **NO Parameters**
- **output dimension**：$(N, D)$  
   $D = C^{l} \times H^{out} \times W^{out}$

**example**：
- input：\( (1, 32, 14, 14) \)
- output：\( (1, 6272) \)


#### F. Fully-connected layer
| parameters   | dimension           | details                     |
|------------|--------------------|--------------------------|
| **weights**   | $(D, D_{out})$ | $D$为输入特征维度    |
| **bias**   | $(D_{out})$         | 每个输出神经元一个偏置    |

**output dimension**：$(N, D_{out})$

**example**：
- input：\( (1, 6272) \)
- output：\( (1, 512) \)


#### G. Output Layer
| parameters   | dimension           | details                     |
|------------|--------------------|--------------------------|
| **weight**   | $(D_{out}, M)$      | $M$为分类类别数           |
| **bias**   | $(M)$              | 每个类别一个偏置          |

**output dimension**：$(N, M)$


**example**（MNIST分类）：
- input：\( (1, 512) \)
- output：\( (1, 10) \)











输入 → Conv1 → ReLU → Pool1 → Flatten → Dense1 → Output


(1,1,28,28) → (1,32,28,28) → (1,32,14,14) → (1,6272) → (1,512) → (1,10)

### 2.2 concise implementation of forward propagation

with the above MNIST example , you can check it using the below code which you are familiar with.

In [ ]:

import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Conv2d(1, 32, 3, padding=1),  # 保持 (28,28)
    nn.ReLU(),
    nn.MaxPool2d(2),                  # 输出 (14,14)
    nn.Flatten(),                     # 32×14×14=6272
    nn.Linear(6272, 512),
    nn.Linear(512, 10)
)

x = torch.randn(1, 1, 28, 28)
print(model(x).shape)  # 输出 torch.Size([1, 10])

here I want to highlight the parameters in Conv2d layer, for more details,You can refer to the parameter definitions on the PyTorch official website, or review Professor Wang's lecture slides. 



In [ ]:
'''
torch.nn.Conv2d(
    in_channels,       # 输入通道数（int）
    out_channels,      # 输出通道数（int）
    kernel_size,       # 卷积核大小（int或tuple）
    stride=1,          # 步长（默认1）
    padding=0,         # 零填充（默认0）
    dilation=1,        # 空洞卷积（默认1）
    groups=1,          # 分组卷积（默认1）
    bias=True,         # 是否使用偏置（默认True）
    padding_mode='zeros', # 填充模式（默认'zeros'）
    device=None,       # 设备（如'cuda'）
    dtype=None         # 数据类型（如torch.float32）
)

'''

# 3-Backward Propagation

We have learnt backpropagation for fully-connected networks,thus, it remains to show the backward propagation for
1) the pooling layer
2) the convolution layer

In the following , we assume a POOL is conducted right after a CONV

### 3.1 Pooling

#### 3.1.1  max pooling


1. Basic Concept

    
For a 4×4 matrix with 2×2 pooling window (stride=2):

$$
A^{l} = \begin{bmatrix} 
1 & 3 & 2 & 1 \\ 
4 & 2 & 0 & 5 \\ 
3 & 1 & 4 & 2 \\ 
2 & 0 & 3 & 6 
\end{bmatrix}
$$


2. Forward Pass
Divide input into non-overlapping 2×2 regions and take maximum values:

| Position       | Matrix                     | Max Value |
|----------------|----------------------------|-----------|
| Top-left       | \begin{pmatrix} 1 & 3 \\ 4 & 2 \end{pmatrix} | **4**     |
| Top-right      | \begin{pmatrix} 2 & 1 \\ 0 & 5 \end{pmatrix}  | **5**     |
| Bottom-left    | \begin{pmatrix} 3 & 1 \\ 2 & 0 \end{pmatrix}  | **3**     |
| Bottom-right   | \begin{pmatrix} 4 & 2 \\ 3 & 6 \end{pmatrix}  | **6**     |


$$
Z^{l} = \begin{bmatrix} 
4 & 5  \\ 
3 & 6  
\end{bmatrix}
$$


3. Backpropagation Principle
Gradients only flow back to positions that were selected during forward pass:

we assume up-streaming gradient to $Z^{l}$ is
$$
∂L/∂Z^{l} = \begin{bmatrix}
0.5 & 1.2 \\
0.8 & -0.3
\end{bmatrix}
$$

then we may derive the gradient to $A^{l}$ is

$$
∂L/∂A^{l} = \begin{bmatrix} 
0.0 & 0.0 & 0.0 & 1.2 \\ 
0.5 & 0.0 & 0.0 & 0.0 \\ 
0.8 & 0.0 & 0.0 & 0.0 \\ 
0.0 & 0.0 & 0.0 & -0.3 
\end{bmatrix}
$$

对于每个输出元素 $Z^{l}(m,n)$：

1)找到对应输入区域中最大值的位置 (i,j)

2)将 $∂L/∂Z^{l}(m,n)$ 的值赋给 $∂L/∂A^{l}(i,j)$

3)其他所有位置的梯度保持为0

In general,

$$
∂L/∂A^{l} = \begin{bmatrix} 
0 & 0 & 0 & ∂L/∂Z^{l}_{1,2} \\ 
∂L/∂Z^{l}_{1,1} & 0 & 0 & 0 \\ 
∂L/∂Z^{l}_{2,1} & 0 & 0 & 0 \\ 
0 & 0 & 0 & ∂L/∂Z^{l}_{2,2} 
\end{bmatrix}
$$

here we also introduce mask matrix,a mask matrix is a crucial component used to selectively highlight or filter specific elements within a data structure. This concept is particularly useful in operations where certain elements need to be ignored or emphasized based on specific criteria.

in this example,

$$
M_{1,1} = \begin{bmatrix} 
0 & 0  \\ 
1 & 0   
\end{bmatrix}
$$

$$
M_{1,2} = \begin{bmatrix} 
0 & 1  \\ 
0 & 0 
\end{bmatrix}
$$

$$
M_{2,1} = \begin{bmatrix} 
1 & 0  \\ 
0 & 0 
\end{bmatrix}
$$

$$
M_{2,2} = \begin{bmatrix} 
0 & 0 \\ 
0 & 1
\end{bmatrix}
$$

$$
∂L/∂A^{l} = \begin{bmatrix} 
M_{1,1}*∂L/∂Z^{l}_{1,1} & M_{1,2}*∂L/∂Z^{l}_{1,2}  \\ 
M_{2,1}*∂L/∂Z^{l}_{2,1} &  M_{2,2}*∂L/∂Z^{l}_{2,2}
\end{bmatrix}
$$


4. code for forward,backward propagation (max_pooling senario)

In [ ]:
import numpy as np

class MaxPool2D:
    def __init__(self, pool_size=2, stride=2):
        self.pool_size = pool_size if isinstance(pool_size, tuple) else (pool_size, pool_size)
        self.stride = stride
        self.input = None
        self.max_indices = None  
    
    def forward(self, x):
        """forward propagation"""
        self.input = x
        batch_size, channels, in_h, in_w = x.shape
        out_h = (in_h - self.pool_size[0]) // self.stride + 1
        out_w = (in_w - self.pool_size[1]) // self.stride + 1

        
        output = np.zeros((batch_size, channels, out_h, out_w))
        
        
        self.max_indices = np.zeros((batch_size, channels, out_h, out_w, 2), dtype=int)
        
        for b in range(batch_size):
            for c in range(channels):
                for h in range(out_h):
                    for w in range(out_w):
                        h_start = h * self.stride
                        h_end = h_start + self.pool_size[0]
                        w_start = w * self.stride
                        w_end = w_start + self.pool_size[1]
                        
                        region = x[b, c, h_start:h_end, w_start:w_end]
                        output[b, c, h, w] = np.max(region)
                        
                        
                        max_pos = np.unravel_index(np.argmax(region), region.shape)
                        self.max_indices[b, c, h, w] = [
                            h_start + max_pos[0],  
                            w_start + max_pos[1]   
                        ]
        return output

### YOUR CODE BEGINS HERE ###
    def backward(self, dout):
        """backward propagation"""
        dx = np.zeros_like(self.input)
        batch_size, channels, out_h, out_w = dout.shape
        
        for b in range(batch_size):
            for c in range(channels):
                for h in range(out_h):
                    for w in range(out_w):
                        max_h, max_w = self.max_indices[b, c, h, w]
                        dx[b, c, max_h, max_w] = dout[b, c, h, w]
        return dx

### YOUR CODE ENDS HERE ###


Below is a test,using which you can verify whether your backward propagation code is right!

In [ ]:
##Please do not change this code###
def numerical_gradient_check():
    """数值梯度检查"""
    np.random.seed(42)
    
   
    x = np.random.randn(1, 1, 4, 4)
    pool = MaxPool2D(pool_size=2, stride=2)
    
   
    out = pool.forward(x)
    
   
    def loss_func(x):
        return np.sum(pool.forward(x))
    
    
    dout = np.ones_like(out)  
    dx_analytic = pool.backward(dout)
    
    
    dx_numeric = np.zeros_like(x)
    h = 1e-5  # 微小扰动
    
    for b in range(x.shape[0]):
        for c in range(x.shape[1]):
            for i in range(x.shape[2]):
                for j in range(x.shape[3]):
                    
                    orig_val = x[b, c, i, j].copy()
                    
                    
                    x[b, c, i, j] = orig_val + h
                    loss_plus = loss_func(x)
                    
                    
                    x[b, c, i, j] = orig_val - h
                    loss_minus = loss_func(x)
                    
                    
                    dx_numeric[b, c, i, j] = (loss_plus - loss_minus) / (2 * h)
                    
                    
                    x[b, c, i, j] = orig_val
    
    # 比较梯度的差异
    diff = np.abs(dx_analytic - dx_numeric)
    print("数值梯度检查结果:")
    print(f"最大差异: {np.max(diff):.2e}")
    print(f"平均差异: {np.mean(diff):.2e}")
    
    # 可视化比较
    print("\n解析梯度 (dx_analytic):")
    print(dx_analytic[0,0])
    print("\n数值梯度 (dx_numeric):")
    print(dx_numeric[0,0])
    
    # 检查是否通过测试
    assert np.allclose(dx_analytic, dx_numeric, rtol=1e-5, atol=1e-5), "梯度检查未通过!"
    print("\n梯度检查通过!")




In [ ]:
# you can also provide your own example,give it a try!
###your input
###forward propagation 
###backward propagation 
###gradient check

###your code begins here###

    x = np.array([[
        [[1, 3, 2, 1],
         [4, 2, 0, 5],
         [3, 1, 4, 2],
         [2, 0, 3, 6]]
    ]], dtype=np.float32)
    
    pool = MaxPool2D(pool_size=2, stride=2)
    
    print("输入矩阵:")
    print(x[0,0])
    
    # 前向传播
    out = pool.forward(x)
    print("\n前向传播输出:")
    print(out[0,0])
    
    # 反向传播
    dout = np.ones_like(out)  # 假设上游梯度全为1
    dx = pool.backward(dout)
    print("\n反向传播梯度:")
    print(dx[0,0])
    
    # 数值梯度检查
    print("\n正在进行数值梯度检查...")
    numerical_gradient_check()
###YOUR CODE ENDS HERE###

#### 3.1.2 average pooling


1.with the same $A^{l}$ as mentioned,here we can derive the $Z^{l}$ 


| Region          | Elements Included | Calculation          | Output Value |
|-----------------|-------------------|----------------------|--------------|
| Top-left (0-1,0-1) | 1, 3, 4, 2        | (1+3+4+2)/4 = 10/4  | 2.5          |
| Top-right (0-1,2-3) | 2, 1, 0, 5       | (2+1+0+5)/4 = 8/4   | 2.0          |
| Bottom-left (2-3,0-1) | 3, 1, 2, 0      | (3+1+2+0)/4 = 6/4   | 1.5          |
| Bottom-right (2-3,2-3) | 4, 2, 3, 6     | (4+2+3+6)/4 = 15/4  | 3.75         |


$$
Z^{l} = \begin{bmatrix} 
2.5 & 2.0  \\ 
1.5 & 3.75  
\end{bmatrix}
$$

2.For Average Pooling, the upstream gradients are evenly distributed to all positions in the corresponding region during the forward pass.

we also assume up-streaming gradient to $Z^{l}$ is
$$
∂L/∂Z^{l} = \begin{bmatrix}
0.5 & 1.2 \\
0.8 & -0.3
\end{bmatrix}
$$

then we may derive the gradient to $A^{l}$ is

$$
∂L/∂A^{l} = \begin{bmatrix} 
0.125 & 0.125 & 0.300 & 0.300 \\ 
0.125 & 0.125 & 0.300 & 0.300 \\ 
0.200 & 0.200 & -0.075 & -0.075 \\ 
0.200 & 0.200 & -0.075 & -0.075 
\end{bmatrix}
$$

$∂L/∂A^{l}_{i,j} = ∂L/∂Z^{l}_{m,n}$ × (1 / pool_area)

here pool_area = 2×2 = 4

3.We can also use mask matrix to simplify the representation

$$
M = \begin{bmatrix} 
1/4 & 1/4  \\ 
1/4 & 1/4  
\end{bmatrix}
$$


$$
∂L/∂A^{l} = \begin{bmatrix} 
M*∂L/∂Z^{l}_{1,1} & M*∂L/∂Z^{l}_{1,2}  \\ 
M*∂L/∂Z^{l}_{2,1} &  M*∂L/∂Z^{l}_{2,2}
\end{bmatrix}
$$

4. code for backward propagation(avg_pooling senario)

Here we can code the average-pooling backward process!

In [ ]:
import numpy as np

class AvgPool2D:
    def __init__(self, pool_size=2, stride=2):
        self.pool_size = pool_size if isinstance(pool_size, tuple) else (pool_size, pool_size)
        self.stride = stride
        self.input_shape = None
    
    def forward(self, x):
        """forward propagation"""
        self.input_shape = x.shape
        batch_size, channels, in_h, in_w = x.shape
        out_h = (in_h - self.pool_size[0]) // self.stride + 1
        out_w = (in_w - self.pool_size[1]) // self.stride + 1
        
        output = np.zeros((batch_size, channels, out_h, out_w))
        
        for b in range(batch_size):
            for c in range(channels):
                for h in range(out_h):
                    for w in range(out_w):
                        h_start = h * self.stride
                        h_end = h_start + self.pool_size[0]
                        w_start = w * self.stride
                        w_end = w_start + self.pool_size[1]
                        
                        region = x[b, c, h_start:h_end, w_start:w_end]
                        output[b, c, h, w] = np.mean(region)
        return output

    
   
    
    ### YOUR CODE BEGINS HERE ###
    def backward(self, dout):
        """backward propagation"""
        dx = np.zeros(self.input_shape)
        batch_size, channels, out_h, out_w = dout.shape
        pool_area = self.pool_size[0] * self.pool_size[1]
        
        for b in range(batch_size):
            for c in range(channels):
                for h in range(out_h):
                    for w in range(out_w):
                        h_start = h * self.stride
                        h_end = h_start + self.pool_size[0]
                        w_start = w * self.stride
                        w_end = w_start + self.pool_size[1]
                        
                        # 将梯度均匀分配到对应区域
                        dx[b, c, h_start:h_end, w_start:w_end] += dout[b, c, h, w] / pool_area
        return dx
    ###YOUR CODE ENDS HERE###


Below is a test,using which you can verify whether your backward propagation code is right!

In [ ]:
### Please do not change this code ###
def numerical_gradient_check():
    """数值梯度检查"""
    np.random.seed(42)
    
    
    x = np.random.randn(1, 1, 4, 4)
    pool = AvgPool2D(pool_size=2, stride=2)
    
    
    out = pool.forward(x)
    
    def loss_func(x):
        return np.sum(pool.forward(x))
    
    
    dout = np.ones_like(out)  
    dx_analytic = pool.backward(dout)
    
    
    dx_numeric = np.zeros_like(x)
    h = 1e-5  # 微小扰动
    
    for b in range(x.shape[0]):
        for c in range(x.shape[1]):
            for i in range(x.shape[2]):
                for j in range(x.shape[3]):
                    
                    orig_val = x[b, c, i, j].copy()
                    
                    
                    x[b, c, i, j] = orig_val + h
                    loss_plus = loss_func(x)
                    
                    
                    x[b, c, i, j] = orig_val - h
                    loss_minus = loss_func(x)
                    
                    
                    dx_numeric[b, c, i, j] = (loss_plus - loss_minus) / (2 * h)
                    
                    
                    x[b, c, i, j] = orig_val
    
    # 比较梯度的差异
    diff = np.abs(dx_analytic - dx_numeric)
    print("数值梯度检查结果:")
    print(f"最大差异: {np.max(diff):.2e}")
    print(f"平均差异: {np.mean(diff):.2e}")
    
    # 可视化比较
    print("\n解析梯度 (dx_analytic):")
    print(dx_analytic[0,0])
    print("\n数值梯度 (dx_numeric):")
    print(dx_numeric[0,0])
    
    # 检查是否通过测试
    assert np.allclose(dx_analytic, dx_numeric, rtol=1e-5, atol=1e-5), "梯度检查未通过!"
    print("\n梯度检查通过!")




In [ ]:
# you can also provide your own example,give it a try!

###your input
###forward propagation 
###backward propagation 
###gradient check

###your code begins here###

    x = np.array([[
        [[1, 3, 2, 1],
         [4, 2, 0, 5],
         [3, 1, 4, 2],
         [2, 0, 3, 6]]
    ]], dtype=np.float32)
    
    pool = AvgPool2D(pool_size=2, stride=2)
    
    print("输入矩阵:")
    print(x[0,0])
    
    # 前向传播
    out = pool.forward(x)
    print("\n前向传播输出:")
    print(out[0,0])
    
    # 反向传播
    dout = np.ones_like(out)  # 假设上游梯度全为1
    dx = pool.backward(dout)
    print("\n反向传播梯度:")
    print(dx[0,0])
    
    # 数值梯度检查
    print("\n正在进行数值梯度检查...")
    numerical_gradient_check()

### 3.2 Convolution

The convolutional layer consists of convolutional kernels and activation functions.

#### Forward Propagation Order
Convolution operation → Activation function (e.g., sigmoid)
#### Backward Propagation Order（using the Chain Rule）
Gradient of activation function → Gradient of convolution operation

#### 3.2.1 backward propagation for activation function

Here we briefly review the backward propagation of activation function, you can check the logistic regression slide for the detailed gradient. The activation function is applied element-wise during the forward pass, so during backpropagation, its gradient is computed as the gradient from the upstream layer multiplied by the derivative of the activation function at each element.

For an activation function $\sigma(\cdot)$ and upstream gradient $\frac{\partial L}{\partial Y}$, the gradient w.r.t. the input $X$ is:

$$\frac{\partial L}{\partial X} = \frac{\partial L}{\partial Y} \odot \sigma'(X)$$

where $\odot$ denotes element-wise multiplication, and $\sigma'(X)$ is the derivative of $\sigma$ evaluated at the original input $X$.

#### 3.2.2 forward propagation for convolution operation

Applying convolutional kernels (also known as filters) to the input data. This process extracts features from the input, which can then be used by subsequent layers in the network. 

Here's a step-by-step explanation:

#### 1.Input Data:
The input to the convolutional layer is typically a multi-dimensional array (tensor). For example, in image processing, the input might be a 3D tensor with dimensions [height, width, channels] (e.g., a 28x28x3 image for RGB).

#### 2.Convolutional Kernels (Filters):
Convolutional kernels are small matrices that slide over the input data to perform the convolution operation.  Each kernel is designed to detect specific features in the input data .
For example, if the input is a 28x28x3 image, a kernel might be a 3x3x3 matrix. The kernel size (3x3) defines the spatial extent, and the depth (3) matches the number of input channels(very important!).

#### 3.Convolution Process:
The convolution operation involves sliding the kernel over the input data and computing the dot product between the kernel and the corresponding region of the input.
Specifically, for each position of the kernel on the input, the element-wise multiplication is performed between the kernel and the input region, and then the results are summed to produce a single output value.
This process is repeated for every position of the kernel on the input, resulting in a 2D output map (called a feature map) for each kernel.

#### 4.Multiple Kernels and Feature Maps:
Typically, multiple kernels are used in a convolutional layer. Each kernel produces a different feature map.
If there are K kernels, the output will be a 3D tensor with dimensions [output height, output width, K], where each channel corresponds to a feature map generated by one of the kernels.

#### 5.Bias Term:
A bias term is often added to each feature map after the convolution operation. This bias term is a scalar value that is added to each element of the feature map.
The bias helps in shifting the activation function and provides additional flexibility in learning.

#### 6.Output Calculation:
For each position (i,j) in the output feature map, the value is computed as:
$$ Output(i,j) = ∑(Kernel(m,n,c) * Input(i+m,j+n,c)) + Bias $$
where (m,n) are the spatial indices of the kernel, and c is the channel index.

#### 3.2.3 Backward propagation for the bias

$$\frac{\partial L}{\partial b} = \sum_{i,j} \frac{\partial L}{\partial X(i,j)}$$

here b denotes the bias , the partial derivative denotes the upstream gradient(activation function)

#### 3.2.4 Backward propagation for the convolution kernel

$$ Z^{l-1} → Z^{l}_{0} $$

Here $ Z^{l-1} $denotes the output of $l-1$ layer, $Z^{l}_{0}$ denotes the output after the convolution kernel.

for simplicity , we consider one kernel.



##### gradient towards $Z^{l-1}$ 

$$
\frac{\partial L}{\partial Z^{l-1}[p,q]} = 
\sum_{i,j} \frac{\partial L}{\partial Z^l_0[i,j]} \cdot 
\frac{\partial Z^l_0[i,j]}{\partial Z^{l-1}[p,q]}
$$

##### gradient towards $W$ 

$$
\frac{\partial L}{\partial W[m,n]} = 
\sum_{i,j} \frac{\partial L}{\partial Z^l_0[i,j]} \cdot 
\frac{\partial Z^l_0[i,j]}{\partial W[m,n]}
$$

##### notation

- $L$: loss function
- $Z^{l-1}$: input of the ${l^{th}}$ layer
- $Z^l_0$: output of the kernel operation（before bias term,activation function）
- $W$: kernel
- $i,j$: coordinate index
- $p,q$: coordinate index
- $m,n$: coordinate index

Just remember: to find derivative with respect to something , we need to find where its information is contained.
It is obvious that every element in $Z^{l}_{0}$ contains information of W(the kernel)

# 4- forward ,Backward propagation code for convolution layer

### 4.1 forward propagation

In [ ]:
import numpy as np

def conv_forward(X, W, b, stride=1, pad=0, activation='relu'):
    """
    forward propagation for convolution layer 
    parameters:
        X: input (n, C, H_in, W_in)
        W: kernel (F, C, HH, WW)
        b: bias (F,)
        stride: 
        pad: 
        activation: activation function ('relu'/'sigmoid'/'tanh'/None)
    return:
        out: output (n, F, H_out, W_out)
        cache: 
    """



    ### YOUR CODE BEGINS HERE ###
    n, C, H_in, W_in = X.shape
    F, _, HH, WW = W.shape
    
    # calculate the shape of output
    H_out = int((H_in + 2*pad - HH)/stride) + 1
    W_out = int((W_in + 2*pad - WW)/stride) + 1
    
    #padding
    X_pad = np.pad(X, ((0,0), (0,0), (pad,pad), (pad,pad)), mode='constant')
    
    # calculation
    out = np.zeros((n, F, H_out, W_out))
    for i in range(n):
        for f in range(F):
            for h in range(H_out):
                for w in range(W_out):
                    h_start = h * stride
                    h_end = h_start + HH
                    w_start = w * stride
                    w_end = w_start + WW
                    
                    out[i,f,h,w] = np.sum(
                        X_pad[i,:,h_start:h_end,w_start:w_end] * W[f]
                    ) + b[f]
    
    # activation function
    if activation == 'relu':
        out = np.maximum(0, out)
    elif activation == 'sigmoid':
        out = 1/(1+np.exp(-out))
    elif activation == 'tanh':
        out = np.tanh(out)
    
    cache = (X, W, b, stride, pad, X_pad, out, activation)
    return out, cache
    ### YOUR CODE ENDS HERE ###


### 4.2 backward propagation

In [ ]:
### Your code begins here ###

def conv_backward(dout, cache):
    """
    backward propagation for convolution layer
    parameters:
        dout: upstream gradient (n, F, H_out, W_out)
        cache: forward cache
    return:
        dX: gradient of input (n, C, H_in, W_in)
        dW: gradient of kernel (F, C, HH, WW)
        db: gradient of bias (F,)
    """
    
    X, W, b, stride, pad, X_pad, Z, activation = cache
    
    # gradient for activation function
    if activation == 'relu':
        dout = dout * (Z > 0)
    elif activation == 'sigmoid':
        dout = dout * Z * (1 - Z)
    elif activation == 'tanh':
        dout = dout * (1 - Z**2)
    
    n, C, H_in, W_in = X.shape
    F, _, HH, WW = W.shape
    _, _, H_out, W_out = dout.shape
    
    # Initialize the gradient
    dX = np.zeros_like(X)
    dW = np.zeros_like(W)
    db = np.zeros_like(b)
    dX_pad = np.pad(dX, ((0,0), (0,0), (pad,pad), (pad,pad)), mode='constant')
    
    # calculate the gradient
    for i in range(n):
        for f in range(F):
            db[f] += np.sum(dout[i,f])
            for h in range(H_out):
                for w in range(W_out):
                    h_start = h * stride
                    h_end = h_start + HH
                    w_start = w * stride
                    w_end = w_start + WW
                    
                    
                    dW[f] += X_pad[i,:,h_start:h_end,w_start:w_end] * dout[i,f,h,w]
                    
                    
                    dX_pad[i,:,h_start:h_end,w_start:w_end] += W[f] * dout[i,f,h,w]
    
   
    if pad > 0:
        dX = dX_pad[:,:,pad:-pad,pad:-pad]
    else:
        dX = dX_pad
    
    return dX, dW, db

### your code ends here ### 

Here is a test using which you can examine if your backward propagation code is correctly written.

In [ ]:

###Please do not change this code###
def eval_numerical_gradient(f, x, df, h=1e-5):
    """数值计算梯度"""
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        ix = it.multi_index
        
        oldval = x[ix]
        x[ix] = oldval + h
        pos = f(x).copy()
        x[ix] = oldval - h
        neg = f(x).copy()
        x[ix] = oldval
        
        grad[ix] = np.sum((pos - neg) * df) / (2 * h)
        it.iternext()
    return grad

def gradient_check():
    np.random.seed(1)
    X = np.random.randn(1, 1, 5, 5)
    W = np.random.randn(2, 1, 3, 3)
    b = np.random.randn(2)
    stride = 1
    pad = 1
    activation = 'relu'
    
    
    out, cache = conv_forward(X, W, b, stride, pad, activation)
    
   
    dout = np.random.randn(*out.shape)
    
    dX, dW, db = conv_backward(dout, cache)
    
    
    f = lambda W: conv_forward(X, W, b, stride, pad, activation)[0]
    dW_num = eval_numerical_gradient(f, W, dout)
    
    f = lambda b: conv_forward(X, W, b, stride, pad, activation)[0]
    db_num = eval_numerical_gradient(f, b, dout)
    
    f = lambda X: conv_forward(X, W, b, stride, pad, activation)[0]
    dX_num = eval_numerical_gradient(f, X, dout)
    
    # 比较差异
    dW_error = np.max(np.abs(dW - dW_num))
    db_error = np.max(np.abs(db - db_num))
    dX_error = np.max(np.abs(dX - dX_num))
    
    print("dW相对误差:", dW_error/np.max(np.abs(dW_num)))
    print("db相对误差:", db_error/np.max(np.abs(db_num)))
    print("dX相对误差:", dX_error/np.max(np.abs(dX_num)))
    
    # 阈值检查
    threshold = 1e-6
    assert dW_error/np.max(np.abs(dW_num)) < threshold, "dW梯度检验失败"
    assert db_error/np.max(np.abs(db_num)) < threshold, "db梯度检验失败"
    assert dX_error/np.max(np.abs(dX_num)) < threshold, "dX梯度检验失败"
    print("所有梯度检验通过!")

# 运行梯度检验
gradient_check()